In [1]:
import torch, torchvision
import cv2
from torchvision import transforms
from torch import nn
from ultralytics import YOLO
from PIL import Image
from helpers_functions.process_frame import process_frame

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Import models

In [4]:
# Import classifier model
efficentNet_b0_weights = torch.load(r'..\models\efficentNet_b0.pth')
efficient_bo_model = torchvision.models.efficientnet_b0()
efficient_bo_model.classifier = nn.Sequential(
    nn.Dropout(p=0.4, inplace=True),
    nn.Linear(in_features=1280, out_features=7, bias=True)
)
efficient_bo_model.load_state_dict(efficentNet_b0_weights)
classifier = efficient_bo_model.to(device)

classifier_transforms = transforms.Compose([
    transforms.Resize(48),
    transforms.CenterCrop(48),
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop(224),
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])
# Classes to classify
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [5]:
# Import detection model
detector = YOLO(r'..\models\yolo8n_best.pt', task='detect')

## Main pipeline

In [13]:
import cv2

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
if not cap.isOpened():
    print("Error: couldn't open the video.")
    exit()

frame_skip = 3
frame_counter = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("The video is completed.")
        break
        
    frame = cv2.resize(frame, (640, 480))

    if frame_counter % frame_skip == 0:
        processes_frame = process_frame(frame=frame, detector=detector, 
                                        classifier_transforms=classifier_transforms,
                                        classifier=classifier,
                                        device=device,
                                        classes=classes)
    else:
        processes_frame = frame
    
    cv2.imshow('Video', processes_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): # Press 'q' to out
        break

cap.release()
cv2.destroyAllWindows()

30.0


In [41]:
import cv2

# Настройки для захвата видео
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Получаем FPS для настройки записи
fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30  # Если FPS не определён, берём значение по умолчанию
print(f"FPS: {fps}")

if not cap.isOpened():
    print("Error: couldn't open the video.")
    exit()

# Настройки для записи видео
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Кодек (XVID для .avi файлов)
output_file = r"C:\Users\tyuri\Downloads\saved_video.mp4"           # Имя выходного файла
frame_size = (640, 480)                   # Размеры кадра
out = None                                # Инициализация объекта записи

# Счётчик кадров
frame_skip = 1
frame_counter = 0

# Флаг для определения начала записи
recording = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("The video is completed.")
        break

    frame = cv2.resize(frame, frame_size)

    if frame_counter % frame_skip == 0:
        # Здесь ваш процесс обработки кадра (заменить на реальный код)
        processes_frame = process_frame(frame=frame, detector=detector, 
                                        classifier_transforms=classifier_transforms,
                                        classifier=classifier,
                                        device=device,
                                        classes=classes)
    else:
        processes_frame = frame

    # Отображение видео
    cv2.imshow('Video', processes_frame)

    # Управление началом и концом записи с помощью клавиш 'r' и 's'
    key = cv2.waitKey(1) & 0xFF
    if key == ord('r'):  # Нажать 'r' для начала записи
        if not recording:
            print("Recording started...")
            out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)
            recording = True
    elif key == ord('s'):  # Нажать 's' для остановки записи
        if recording:
            print("Recording stopped.")
            recording = False
            out.release()
            out = None
    elif key == ord('q'):  # Нажать 'q' для выхода
        break

    # Записываем кадры в файл, если запись активна
    if recording and out:
        out.write(processes_frame)

    frame_counter += 1

# Освобождение ресурсов
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()

FPS: 30
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.
Recording started...
Recording stopped.


classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']